In [1]:
!pip install accelerate>=0.20.1

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
 !pip install transformers
 !pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.8/764.8 kB 51.8 MB/s eta 0:00:00


In [4]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

In [5]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/final_project_신문요약/data/news_summary_train_dataset.tsv', sep='\t')
df = train_df[:100]

In [6]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [6]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

model_name = 'digit82/kobart-summarization'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# import torch
# from transformers import PreTrainedTokenizerFast
# from transformers import BartForConditionalGeneration
# import torch.nn as nn

# model_name = 'hyunwoongko/kobart'
# tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [14]:
tokenizer('<s>' + "나는 사람이 아니다" + '</s>', padding="max_length", max_length=10, truncation=True)

{'input_ids': [0, 14651, 14959, 26889, 1, 3, 3, 3, 3, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0]}

In [21]:
input_tokenized = []
target_tokenized = []
for i in range(len(df)):

    input_text = df.iloc[i, 0]
    target_text = df.iloc[i, 1]

    input_ids = tokenizer(input_text, padding="max_length", max_length=1024, truncation=True)["input_ids"]
    target_ids = tokenizer('<s>' + target_text + '</s>', padding="max_length", max_length=1024, truncation=True)["input_ids"]

    input_tokenized.append(input_ids)
    target_tokenized.append(target_ids)

In [22]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, input, target):
        self.input = torch.LongTensor(input)
        self.target = torch.LongTensor(target)

    def __len__(self):
        return len(self.input)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input[idx],
            'labels': self.target[idx]
        }


In [ ]:
from sklearn.model_selection import train_test_split

# 학습을 위해 train, test set으로 나눈다.
input_train, input_test, target_train, target_test = train_test_split(input_tokenized, target_tokenized, test_size=0.1, random_state=42)
print(len(input_train), len(input_test), len(target_train), len(target_test))

90 10 90 10


In [ ]:
# from torch.utils.data import DataLoader

# train_dataset = CustomDataset(input_train, target_train)
# test_dataset = CustomDataset(input_test,target_test)

# dl_train = DataLoader(train_dataset, batch_size=1, shuffle=False)
# dl_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [29]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
# from transformers import BartConfig
# import torch.nn as nn

# class SummarizeModel(nn.Module):
#     def __init__(self, model_name = 'hyunwoongko/kobart', num_decode_layers = 1):
#         super(SummarizeModel, self).__init__()
#         self.config = BartConfig.from_pretrained(model_name)
#         self.config.decoder_layers = num_decode_layers
#         self.bart =  BartForConditionalGeneration.from_pretrained(model_name, config = self.config)
#         for param in self.bart.get_encoder().parameters():
#                 param.requires_grad = False

#     def forward(self, input_ids, dec_input):
#         result = self.bart(input_ids, dec_input)

#         return result[0]


#     def generate(self, input_ids):
#         result = self.bart(input_ids)

#         return result[0]

In [8]:
torch.cuda.empty_cache()

In [10]:
df

,passage,summary
0,40억 달러 ‘딜’ 주인공 김봉진 우아한형제들 대표태풍 뒤의 고요함이랄까. 40...,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...
1,시진핑 방한과 새해 한·중관계 전망지난달 23일 베이징에서 열린 한·중 정상회담을 ...,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지...
2,한·중이 사드에 관한 ‘3불(不)’이 ‘약속’인지 ‘입장표명’인지 표현을 놓고 갈...,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...
3,배달의민족이 독일 자본에 매각된 것을 놓고 말들이 많다. 민족 정서를 배반했다며...,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...
4,지난 28일부터 나흘간 진행된 7기 5차 노동당 전원회의에서 북한은 핵 무력 개발의...,"북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장에 ..."
...,...,...
95,채소·과일·콩류·통곡물·견과류·우유 등이 영양밀도가 높고 탄산음료·아이스크림·과자...,짜게 먹는 습관은 과식을 유발하고 지방세포가 커져 싱겁게 먹는 사람들보다 비만율이 높다.
96,기계를 정비할 때 부품 크기에 맞는 공구를 사용해야 완벽하게 수리할 수 있듯이 맞...,"맞춤형 인공관절 수술은 기존 인공관절 수술 방식과 대비해 높은 수술 정확도, 짧은 ..."
97,새해 첫 주말인 4일에도 두 개의 ‘광장’이었다. 서울 서초동 대검찰청 앞에선 ...,서울 서초동 대검찰청 앞에서 열린 서초달빛집회는 조 전 법무부장관 수호를 결의하는 ...
98,"도널드 트럼프 미국 대통령이 4일(현지시간) ""이란이 미국인을 공격할 경우 아주 중...",트럼프 대통령은 이란이 미국인을 비롯한 미국 재산에 피해를 줄 경우 이란의 고위급 ...


In [23]:
input_train, input_test, target_train, target_test = input_tokenized[:90], input_tokenized[90:100], target_tokenized[:90], target_tokenized[90:100]

train_dataset = CustomDataset(input_train, target_train)
test_dataset = CustomDataset(input_test,target_test)

In [20]:
train_dataset[0]

{'input_ids': '40억 달러 ‘딜’ 주인공 김봉진 우아한형제들 대표태풍 뒤의 고요함이랄까.   40억 달러짜리 ‘딜’(거래)을 마친 뒤의 사무실은 조용했다.   음식 배달 앱 ‘배달의민족’ 운영사 ㈜우아한형제들의 서울 송파구 방이동 사옥은 비어 있었다.   전 직원들에게 연말 특별 휴가를 선물한 김봉진(44) 대표는 혼자 출근해 남은 일을 처리하고 있었다.   축하한다는 말을 건네자 “이제 시작인 걸요”라는 답이 돌아왔다.   그에게 기업 매각은 단순한 ‘엑시트’(창업 후 지분 매각으로 이익을 실현하는 일)가 아니었다.   성공 스토리 뒤에는 환희의 무게만큼 고민이 자리 잡고 있었다.   독일계 음식 배달서비스업체 DH(딜리버리 히어로)가 평가한 우아한형제들의 기업가치는 약 4조8000억원.   국내 스타트업 M&A 사상 최대 규모다.   앱 하나로 평가받은 기업가치가 GS나 현대건설의 시가총액과 맞먹는다.   ‘매각’이라는 표현을 썼지만, 창업자인 김 대표가 회사를 떠나는 것은 아니다.   오히려 역할이 커졌다.   두 회사가 절반씩 출자해 싱가포르에 세우는 합작법인 ‘우아DH아시아’의 책임자로서 아시아 11개국 사업을 총괄하게 된다.   DH는 우아한형제들의 투자자 지분 87%를 인수하고, 김 대표 등 경영진이 가진 지분 13%는 DH 본사 지분으로 전환하기로 했다.   그렇게 되면 김 대표는 DH 경영진 가운데 개인 최대 주주가 된다.    “더 큰 꿈 위해 글로벌 자본 선택” 김 대표의 고민은 ‘민족’이라는 단어에 닿아 있었다.   회사가 외국 자본에 넘어가면서 민족 브랜드가 어울리지 않게 됐다는 시선 때문이다.   소비자 반응이 긍정적이지만은 않다.  “겸허하게 받아들인다.   DH와는 경쟁 관계이지만 창업 초기부터 지속해서 교류해왔다.   그 과정에서 그들이 지닌 ‘글로벌 DNA’에 놀랐다.   DH는 홈그라운드 격인 독일 사업마저 네덜란드 기업에 넘기고 글로벌 마케팅을 강화해왔다.   그들과 계속 싸울지, 합쳐서 글로벌 무대로 나갈지 마

In [9]:
import torch
# from transformers import PreTrainedTokenizerFast
# from transformers import BartForConditionalGeneration
# from transformers import BartConfig

# model_name = 'digit82/kobart-summarization'
# config = BartConfig.from_pretrained(model_name)
# config.decoder_layers = 2

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config = config)
for param in model.get_encoder().parameters():
    param.requires_grad = False

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model
)


model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(30000, 768, padding_idx=3)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(30000, 768, padding_idx=3)
      (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
      (layers): ModuleList(
        (0-5): 6 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=

In [26]:
model_path = "/content/drive/MyDrive/final_project_신문요약/model/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps=12, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3,
    logging_steps=12
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [27]:
trainer.train()

Step,Training Loss,Validation Loss
12,4.718600,4.683564
24,4.696400,4.658010
36,4.660800,4.633872
48,4.638400,4.616115
60,4.614200,4.602845


TrainOutput(global_step=60, training_loss=4.6657154083251955, metrics={'train_runtime': 112.1322, 'train_samples_per_second': 4.013, 'train_steps_per_second': 0.535, 'total_flos': 274381406208000.0, 'train_loss': 4.6657154083251955, 'epoch': 5.0})

## ACCURACY찍기!

In [30]:
# 100개 데이터 모델로 돌린 결과 리스트에 담기 : 담긴건 토큰들
result = []
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]
    output = model.generate(input_ids.to(device))
    result.append(output[0])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [31]:
# label토큰에서 pad빼기 귀찮으니까 그냥 다시 토큰화 해버리기, special token없이
target_tokenized = []
for i in range(len(df)):
    target_text = df.iloc[i, 1]
    target_ids = tokenizer(target_text)["input_ids"]
    target_tokenized.append(target_ids)

In [32]:
result[0]

tensor([    2, 14116, 10897, 12335, 15699, 14454, 26882, 11776, 14759, 25990,
        16400, 14809, 14883, 12037, 14136, 12178, 24189, 16084, 16890,     2],
       device='cuda:0')

In [ ]:
int(False)

0

In [33]:
# 결과물에는 [2, 0, ..., 1] 이런식으로 다 되어있어서 스페셜토큰 다 빼주려고 슬라이싱해서 없어버렸습니다. 그리고 반환타입이 텐서인데 그냥 리스트로 바꿔버림
result_new = []
for i in range(100):
    result_new.append(result[i].tolist()[1:-1])

result_new[0]

[14116,
 10897,
 12335,
 15699,
 14454,
 26882,
 11776,
 14759,
 25990,
 16400,
 14809,
 14883,
 12037,
 14136,
 12178,
 24189,
 16084,
 16890]

In [36]:
# count는 예측이랑 정답의 같은 위치에 같은 토큰이 있을경우 +1
# total은 정답 토큰 총 개수
count = 0
total = 0
for i in range(100):
    total += len(target_tokenized[i])
    if len(result_new[i]) > len(target_tokenized[i]):
        for j in range(len(target_tokenized[i])):
            count += int(result_new[i][j] == target_tokenized[i][j])
    else:
        for j in range(len(result_new[i])):
            count += int(result_new[i][j] == target_tokenized[i][j])

print(count/total)

0.033093525179856115


In [ ]:
348/12

29.0

In [ ]:
count

930

In [ ]:
total

3475

In [ ]:
predict_text = []
for i in range(100):
    predict_text.append(tokenizer.decode(result_new[i]))

In [ ]:
df_result = pd.DataFrame({"predict":predict_text})
df_result['labels'] = df.iloc[:, 1]

In [ ]:
df_result.head(10)

,predict,labels
0,40억 달러짜리 ‘딜’을 마친 뒤의 사무실은 텅 비었고 김,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...
1,한·중7년의 한·중관계를 되돌아보며 시진핑 중국 국가주석의 방한이 확정적이라고,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지...
2,사드에 관한 3불이 불이라는 표현을 놓고 중국이 한국을 압박할 수 있는 카드라고,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...
3,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...
4,북한은 7기 5차 노동당 전원회의에서 핵 무력 개발의 주역인 이병철,"북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장에 ..."
5,부산 해운대의 상징이던 5성급 해운대그랜드호텔이 경기 침체와 적자,부산 해운대의 상징이던 그랜드호텔이 경기 침체와 적자 경영으로 폐업이 불가피하다고 ...
6,워런은 샌더스가 심근경색으로 쓰러진 영향과 트럼프식 정치,워런은 샌더스가 심근경색으로 쓰러진 영향과 트럼프의 대척점에 선 신선한 이미지의 정...
7,새해 김 국무위원장은 신년사 육성 연설에서 자신이 약속했던 경제 개발과 대미,북한의 김 국무위원장은 2012년에 집권한 이후 처음으로 신년사 육성 연설에 모습을...
8,국내 이동통신 3사는 산토끼를 잡느라 집토끼를 위한 서비스에,국내 이동통신 3사는 산토끼를 잡느라 집토끼를 위한 서비스에 인색하다는 지적에 대해...
9,출국 금지 등을 조건으로 보석을 허가 받은 카로로스 곤 전 회장이 레바논에,출국 금지 등을 조건으로 보석을 허가 받은 닛산자동차의 전 회장이 유년기를 보낸 레...


In [ ]:
result

[tensor([    2,     0, 15634, 11764, 16186, 18765, 14143, 10016, 15886, 18761,
         14289, 12024, 15554, 19822,  1700, 13185, 14085, 19126, 14116,     2],
        device='cuda:0'),
 tensor([    2,     0, 28287, 15084, 12024, 28287, 19120, 24859, 28793, 25832,
         14361, 14518, 12258, 20849, 25313, 12034, 16053, 12124, 14217,     2],
        device='cuda:0'),
 tensor([    2,     0, 14031, 23464, 15501, 14086, 26403, 14135, 14394, 24361,
         16005, 20895, 21074, 17793, 13594, 14032, 14082, 16885, 14218,     2],
        device='cuda:0'),
 tensor([    2,     0, 22556, 12024, 20986, 12034, 15604, 16327, 11786, 16918,
          9908, 14374, 16005, 16545, 14045, 15155, 14203, 10773, 28782,     2],
        device='cuda:0'),
 tensor([    2,     0, 20407, 14235,  9264, 14144, 12612, 29161, 20798, 18562,
         14800, 23582, 14646, 12024, 14053, 11803, 12037, 20193, 12676,     2],
        device='cuda:0'),
 tensor([    2,     0, 14802, 20992, 15071, 17113, 18470, 14144, 11280,  92

In [ ]:
for i in range(100):
    if len(result)

tensor([    2,     0, 15634, 11764, 16186, 18765, 14143, 10016, 15886, 18761,
        14289, 12024, 15554, 19822,  1700, 13185, 14085, 19126, 14116,     2],
       device='cuda:0')

In [ ]:
tokenizer.decode(output[0], skip_special_tokens=True)

'40억 달러짜리 딜을 마친 우아한형제들의 사무실은 비어 있고'

In [ ]:
df.iloc[0, 1]

'DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 스타트업 사상 최대 규모의 M&A 이후 합작법인 우아DH아시아의 책임자가 된다.'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,7.462800,7.324414


TrainOutput(global_step=570, training_loss=7.447968493009869, metrics={'train_runtime': 518.0577, 'train_samples_per_second': 17.373, 'train_steps_per_second': 1.1, 'total_flos': 1698523250688000.0, 'train_loss': 7.447968493009869, 'epoch': 10.0})

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/models/model_summary.pt")

In [ ]:
input_ids = tokenizer(df.iloc[0, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
generated_text

'독일 배달 앱 ‘배달의민족’의 운영사 (주)우아한형제'

In [ ]:
input_ids = tokenizer(df.iloc[1, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
generated_text

'한중 정상회담을 계기로 시진핑 중국 국가주석의 방한이 확정적으로 확정적으로 이뤄지며'

In [ ]:
result = []
labels = df.iloc[:, 1]
for i in range(100):
    input_ids = tokenizer(df.iloc[i, 0], return_tensors="pt")["input_ids"]

    generated_ids = model.generate(input_ids.to(device))
    # generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    result.append(generated_ids)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
df_tmp = pd.DataFrame({'result':result})
df_tmp['label'] = labels
df_tmp

,result,label
0,독일 배달 앱 ‘배달의민족’의 운영사 (주)우아한형제,DH가 기업가치를 약 4조 8000억 원으로 평가한 우아한형제들의 김 대표는 국내 ...
1,한중 정상회담을 계기로 시진핑 중국 국가주석의 방한이 확정적으로 확정적으로 이뤄지며,한·중관계가 중국 쪽으로 기울어진 비대칭관계로 바뀌면서 세계무대에서 한국의 경제 지...
2,중국이·중·일 관계 악화로 인한 양국 갈등은 한국으로 하여금 전략적 의심을 받고,사드에 관한 3불이 결국은 중국이 한국을 압박할 수 있는 카드이며 미국이 중거리 미...
3,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며,배달의민족이 독일 자본에 매각된 것을 놓고 민족 정서를 배반했다며 분노가 터져 나왔...
4,북한은 노동당 무력 개발의 주역인 이병철 군수공업부 제1부부장을 정치,"북한은 핵 무력 개발의 주역인 이 제1부부장을 정치국 위원과 당 부위원장, 부장에 ..."
...,...,...
95,채 식습관 개선은 나트륨 섭취를 줄이고 국과 찌개를 줄이고,짜게 먹는 습관은 과식을 유발하고 지방세포가 커져 싱겁게 먹는 사람들보다 비만율이 높다.
96,3D프린터 맞춤형 인공관절 수술은 인공관절의 내구성을,"맞춤형 인공관절 수술은 기존 인공관절 수술 방식과 대비해 높은 수술 정확도, 짧은 ..."
97,서초 서초동 대검찰청 앞에서는 검찰개혁 완수를 요구하는 '서초달,서울 서초동 대검찰청 앞에서 열린 서초달빛집회는 조 전 법무부장관 수호를 결의하는 ...
98,트럼프 미국 이란 혁명 당시 444일간 미 대사관에 억류됐던 미국인 인질,트럼프 대통령은 이란이 미국인을 비롯한 미국 재산에 피해를 줄 경우 이란의 고위급 ...


In [ ]:
generated_ids

tensor([[    2,     0, 16621, 11280, 17078, 14188, 15334, 14934, 15143, 29798,
         14379, 15068, 16243, 12037, 20238, 10675, 18078, 24592,   315,     2]],
       device='cuda:0')

In [ ]:
result[4][0]

tensor([    2, 20407, 29161, 23582, 14646, 12024, 14053, 11803, 12037, 20193,
        12676, 14459, 11372, 17610, 10948, 15953, 28808, 14324, 14499,     2],
       device='cuda:0')

In [ ]:
result[0][0][0].item()

2

In [ ]:
target_tokenized[0]

[0,
 15529,
 271,
 8981,
 14413,
 8981,
 14773,
 14383,
 14136,
 12178,
 14260,
 14474,
 11764,
 27523,
 14653,
 13590,
 29689,
 13590,
 13679,
 12147,
 14180,
 14116,
 15699,
 14454,
 26882,
 11776,
 16400,
 14809,
 17067,
 14759,
 25990,
 14412,
 26236,
 17991,
 29689,
 267,
 271,
 16945,
 12024,
 15317,
 14401,
 25529,
 1,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [ ]:
while 3 in

In [ ]:
count = 0

for i in range(100):
    for j in range(len(result[i])):
        if result[i][0][j].item() == target_tokenized[i][j]:
            count += 1

In [ ]:
count

0

In [ ]:
model_path = "/content/drive/MyDrive/hong/models/"

training_args = Seq2SeqTrainingArguments(
    output_dir=model_path, #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    per_device_train_batch_size=64, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps=500, # Number of update steps between two evaluations.
    save_steps=1000, # after # steps model is saved
    warmup_steps=300,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    evaluation_strategy="steps",
    save_total_limit=3
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
500,4.879400,4.836550
1000,4.832300,4.792342
1500,4.793000,4.757692
2000,4.760800,4.729735
2500,4.733400,4.706688
3000,4.712300,4.687441
3500,4.695500,4.671278
4000,4.680400,4.657376
4500,4.666900,4.645577
5000,4.655200,4.635426


KeyboardInterrupt: ignored

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/hong/models/checkpoint-3000/pytorch_model.bin")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
model = model.to(device)

In [ ]:
input_ids = tokenizer(df.iloc[120, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
generated_text

'인공지능이 설계에 대해 인공지능이 구현하는 스마트과 같은 것을 우려하지 않을 수'

In [ ]:
generated_text

'인공지능이 설계에 도입하는 유토피아와 디스토피아를 우려하는 데'

In [ ]:
df.iloc[120, 1]

'인공지능이 금융권, 인터넷 검색, 산업 현장 등의 우리 일상에 깊숙이 들어와 있는 만큼 디스토피아를 우려하지 않을 수 없다. '

In [ ]:
generated_text

'국토교통부가 안전기준을 도입하면서 자동차가 교통상황 전반을 감지할 수 있게 됐'

In [ ]:
df.iloc[110, 1]

'국토부가 제한된 자율주행으로 불리는 레벨3 자율주행차량을 도입하는데 자율주행 단계는 SAE와 NHTSA의 기준에 따른다.'

In [ ]:
input_ids = tokenizer(df.iloc[110, 0], return_tensors="pt")["input_ids"]

generated_ids = model.generate(input_ids.to(device))
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
from transformers import BartConfig
import torch.nn as nn

class SummarizeModel(nn.Module):
    def __init__(self, model_name = 'hyunwoongko/kobart', num_decode_layers = 1):
        super(SummarizeModel, self).__init__()
        self.config = BartConfig.from_pretrained(model_name)
        self.config.decoder_layers = num_decode_layers
        self.bart =  BartForConditionalGeneration.from_pretrained(model_name, config = self.config)
        for param in self.bart.get_encoder().parameters():
                param.requires_grad = False

    def forward(self, input_ids, dec_input):
        result = self.bart(input_ids, dec_input)

        return result[0]


    def generate(self, input_ids):
        result = self.bart(input_ids)

        return result[0]

In [ ]:
model = SummarizeModel().to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Some weights of the model checkpoint at gogamza/kobart-summarization were not used when initializing BartForConditionalGeneration: ['model.decoder.layers.5.encoder_attn.out_proj.weight', 'model.decoder.layers.4.self_attn.v_proj.weight', 'model.decoder.layers.1.self_attn_layer_norm.bias', 'model.decoder.layers.2.encoder_attn.v_proj.weight', 'model.decoder.layers.5.self_attn.v_proj.bias', 'model.decoder.layers.3.final_layer_norm.weight', 'model.decoder.layers.2.self_attn.k_proj.bias', 'model.decoder.layers.1.self_attn.q_proj.bias', 'model.decoder.layers.3.self_attn.q_proj.bias', 'model.decoder.layers.4.fc1.bias', 'model.decoder.layers.4.self_attn.out_proj.bias', 'model.decoder.layers.1.encoder_attn_layer_norm.weight', 'model.decoder.layers.4.encoder_attn.k_proj.bias', 'model.decoder.layers.4.fc1.weight', 'model.decoder.layers.4.encoder_attn.q_proj.bias', 'model.decoder.layers.3.encoder_attn.q_proj.weight', 'model.decoder.layers.1.self_attn.k_proj.weight', 'model.decoder.layers.4.encoder_

In [ ]:
model

SummarizeModel(
  (bart): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(30000, 768, padding_idx=3)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(30000, 768, padding_idx=3)
        (embed_positions): BartLearnedPositionalEmbedding(1028, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bia

In [ ]:
result = model.generate(next(iter(dl_train))['input_ids'].to(device))

In [ ]:
result.shape

torch.Size([1, 512, 30000])

In [ ]:
# from tqdm import tqdm
# epochs = 100
# loss = nn.CrossEntropyLoss().to(device)
# optim = torch.optim.Adam(params=model.parameters(), lr=0.001)

# for epoch in tqdm(range(epochs)):
#     epoch_loss = 0.0
#     for batch in dl_train:
#         input, output = batch['input_ids'].to(device), batch['labels'].to(device)
#         pred = model(input, output)
#         pred = pred.transpose(0, 1)
#         output = output.transpose(0, 1)
#         batch_loss = 0.0
#         for i in range(512):  # input_size만큼 돌기
#             batch_loss += loss(pred[i], output[i])
#         batch_loss = batch_loss/512

#         epoch_loss += batch_loss

#     epoch_loss = epoch_loss/len(dl_train)
#     if (epoch+1) % 10 == 0:
#       print('Epoch:', '%02d' % (epoch + 1), 'cost =', '{:.6f}'.format(epoch_loss))

#     optim.zero_grad()
#     epoch_loss.backward()
#     optim.step()

#     with torch.no_grad():
#         epoch_val_loss = 0.0
#         for batch_val in dl_test:
#             input_val, output_val = batch_val['input_ids'].to(device), batch_val['labels'].to(device)
#             pred_val = model.generate(input_val)
#             pred_val = pred_val.transpose(0, 1)
#             output_val = output_val.transpose(0, 1)

#             batch_loss_val = 0.0
#             for i in range(512):
#                 batch_loss_val += loss(pred_val[i], output_val[i])

#             batch_loss_val = batch_loss_val/512

#         epoch_val_loss = epoch_val_loss/len(dl_test)
#         if (epoch+1) % 10 == 0:
#             print('validation loss =', '{:.6f}'.format(batch_loss_val))

  0%|          | 0/100 [00:15<?, ?it/s]


OutOfMemoryError: ignored

In [ ]:
torch.save(model, "/content/drive/MyDrive/hong/model_simple.pt")